### DS 10 ZEN

In [1]:
import numpy as np
import pandas as pd
import sqlite3

### Task 1
Давай начнем с самого простого. В этом упражнении тебе необходимо получить отфильтрованные данные из таблицы 
в базе данных. Почему важно фильтровать данные именно в запросе, а не после этого в библиотеке Pandas?
Потому что таблицы могут иметь огромный размер. Если ты попыташься получить всю таблицу, то не сможешь ее "переварить" 
– у тебя может не хватить оперативной памяти. Всегда помни об этом.

Первый способ фильтрации — выбрать только те столбцы, которые тебе действительно нужны. 
Второй способ — выбрать только те строки, которые тебе действительно нужны.

Подробное описание задания:

1. Помести базу данных в подкаталог data в корневом каталоге, используемом в рамках этого проекта.
2. Создай соединение с базой данных с помощью библиотеки `sqlite3`.
3. Получи схему таблицы `pageviews` с помощью `pd.io.sql.read_sql` и запроса `PRAGMA table_info(pageviews);`.
4. Получи только первые `10` строк таблицы `pageviews`, чтобы проверить, как она выглядит.
5. Получи подтаблицу с помощью только **одного** запроса, в котором:
    - используются только `uid` и `datetime`;
    - используются только данные пользователей (`user_*`), и не используются данные администраторов;
    - сортировка выполняется по `uid` в порядке возрастания;
    - столбец индекса — это `datetime`;
    - `datetime` преобразован в `DatetimeIndex`;
    - имя датафрейма — `pageviews`.
6. Закрой соединение с базой данных.

In [2]:
# создаем соединение с бд
connection = sqlite3.connect('checking-logs.sqlite')

# получаем схему таблицы pageviews
scheme_q = "PRAGMA table_info(pageviews)"
scheme_t = pd.io.sql.read_sql(scheme_q, connection)
display(scheme_t)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,datetime,TIMESTAMP,0,None,0


In [3]:
# получаем первые 10 строк таблицы pageviews
query = "SELECT * FROM pageviews LIMIT 10"
pageviews_data = pd.io.sql.read_sql(query, connection)
display(pageviews_data)

,index,uid,datetime
0,0,admin_1,2020-04-17 12:01:08.463179
1,1,admin_1,2020-04-17 12:01:23.743946
2,2,admin_3,2020-04-17 12:17:39.287778
3,3,admin_3,2020-04-17 12:17:40.001768
4,4,admin_1,2020-04-17 12:27:30.646665
5,5,admin_1,2020-04-17 12:35:44.884757
6,6,admin_1,2020-04-17 12:35:52.735016
7,7,admin_3,2020-04-17 12:36:21.401412
8,8,admin_3,2020-04-17 12:36:22.023355
9,9,admin_1,2020-04-17 13:55:19.129243


In [4]:
# получаем подтаблицу с условиями
subq = "SELECT uid, datetime FROM pageviews WHERE uid LIKE 'user_%' ORDER BY uid ASC"
pageviews = pd.io.sql.read_sql(subq, connection).rename(columns={'datetime':'DatetimeIndex'}).set_index('DatetimeIndex')
display(pageviews.info())
display(pageviews)

<class 'pandas.core.frame.DataFrame'>
Index: 987 entries, 2020-04-26 21:53:59.624136 to 2020-05-21 16:36:40.915488
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   uid     987 non-null    object
dtypes: object(1)
memory usage: 15.4+ KB


None

,uid
DatetimeIndex,
2020-04-26 21:53:59.624136,user_1
2020-04-26 22:06:19.478143,user_1
2020-04-26 22:12:09.614497,user_1
2020-04-30 19:29:01.831635,user_1
2020-05-05 20:26:32.894852,user_1
...,...
2020-04-29 16:51:21.877630,user_30
2020-05-09 20:30:47.034282,user_30
2020-05-22 11:30:18.368990,user_5


In [5]:
# закрываем соединение с бд
connection.close()

### Task 2
А теперь давай усложним задачу.Ты знаешь, что такое подзапрос? Это как запрос, но только внутри запроса. 
Чем он может быть полезен? Например, с его помощью можно сделать определенную агрегацию для выбранных данных. 
Однако следует помнить, что в первую очередь выполняются вложенные запросы, и только после этого — основной запрос.

Что тебе нужно сделать:

1. Создай соединение с базой данных с помощью библиотеки sqlite3.
2. Получи схему таблицы `checker`.
3. Получи только первые `10` строк таблицы `checker`, чтобы проверить, как она выглядит.
4. Подсчитай, сколько строк удовлетворяют следующим условиям, используя только **один** запрос с любым количеством подзапросов.
    - Подсчитай количество строк из таблицы `pageviews`, но только с пользователями из таблицы `checker`, у которых:
        - `status = 'ready'` (мы не хотим анализировать логи со статусом `checking`);
        - `numTrials = 1` (мы хотим проанализировать только первые коммиты, поскольку только они информируют о том, когда студент начал работать над лабораторным заданием);
        - названия лабораторных заданий должны быть только из следующего списка: `laba04`, `laba04s`, `laba05`, `laba06`, `laba06s`, `project1`. Только они были активными во время проведения эксперимента.
    - Сохрани их в датафрейме `checkers` в столбце `cnt`. Датафрейм будет представлять собой только одну ячейку.
5. Закрой соединение.

In [6]:
# создаем соединение с бд
connection = sqlite3.connect('checking-logs.sqlite')

# получаем схему таблицы checker
scheme_q = "PRAGMA table_info(checker)"
scheme_t = pd.io.sql.read_sql(scheme_q, connection)
display(scheme_t)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,status,TEXT,0,None,0
2,2,success,INTEGER,0,None,0
3,3,timestamp,TIMESTAMP,0,None,0
4,4,numTrials,INTEGER,0,None,0
5,5,labname,TEXT,0,None,0
6,6,uid,TEXT,0,None,0


In [7]:
# получаем первые 10 строк таблицы pageviews
query = "SELECT * FROM checker LIMIT 10"
pageviews_data = pd.io.sql.read_sql(query, connection)
display(pageviews_data)

,index,status,success,timestamp,numTrials,labname,uid
0,0,checking,0,2020-04-16 21:12:50.740474,5,None,admin_1
1,1,ready,0,2020-04-16 21:12:54.708365,5,code_rvw,admin_1
2,2,checking,0,2020-04-16 21:46:47.769088,7,None,admin_1
3,3,ready,0,2020-04-16 21:46:48.121217,7,lab02,admin_1
4,4,checking,0,2020-04-16 21:53:01.862637,6,code_rvw,admin_1
5,5,ready,0,2020-04-16 21:53:05.373389,6,code_rvw,admin_1
6,6,checking,0,2020-04-17 05:18:51.965864,1,None,None
7,7,ready,0,2020-04-17 05:19:02.744528,1,project1,user_4
8,8,checking,0,2020-04-17 05:22:35.249331,2,project1,user_4
9,9,ready,1,2020-04-17 05:22:45.549397,2,project1,user_4


In [8]:
# подсчитываем количество строк, удовлетворяющих условиям
count_q = """
SELECT COUNT() as cnt
FROM pageviews
WHERE uid IN (SELECT uid FROM checker WHERE status = 'ready' AND numTrials = 1 AND labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1'))
"""
checkers = pd.io.sql.read_sql(count_q, connection)
display(checkers)

,cnt
0,985


In [9]:
# закрываем соединение
connection.close()

### Task 3
В этом упражнении ты создашь так называемую витрину данных. Она представляет собой таблицу, 
которую можно использовать для аналитических целей. Обычно она создается путем объединения нескольких отдельных таблиц. 
В этом упражнении мы будем собирать различные данные о наших пользователях: когда они сделали свои первые коммиты, 
когда они впервые посетили ленту новостей и т. д. Это поможет позднее выполнить анализ данных.

Что тебе нужно сделать в этом упражнении (ознакомься с полным описанием задания):

1. Создай соединение с базой данных с помощью библиотеки `sqlite3`.
2. Создай новую таблицу `datamart` в базе данных, объединив таблицы `pageviews` и `checker` с помощью только **одного** запроса.
    - Таблица должна содержать следующие столбцы: `uid`, `labname`, `first_commit_ts`, `first_view_ts`.
    - `first_commit_ts` — это просто новое имя для столбца `timestamp` из таблицы `checker`; он показывает первый коммит конкретного лабораторного задания конкретного пользователя.
    - `first_view_ts` — первое посещение пользователем из таблицы `pageviews`, метка времени посещения пользователем ленты новостей.
    - По-прежнему нужно использовать фильтр `status = 'ready'`.
    - По-прежнему нужно использовать фильтр `numTrials = 1`.
    - Имена лабораторных заданий по-прежнему должны быть из следующего списка: `laba04`, `laba04s`, `laba05`, `laba06`, `laba06s`, `project1`.
    - Таблица должна содержать только пользователей (`uid` с `user_*`), а не администраторов.
    - `first_commit_ts` и `first_view_ts` должны быть распарсены как `datetime64[ns]`.
3. Используя методы библиотеки Pandas, создай два датафрейма: `test` и `control`.
    - `test` должен включать пользователей, у которых имеются значения в `first_view_ts`.
    - `control` должен включать пользователей, у которых отсутствуют значения в `first_view_ts`.
    - Замени пропущенные значения в `control` средним значением `first_view_ts` пользователей из `test` (оно пригодится нам для анализа в будущем).
    - Сохрани обе таблицы в базе данных (вы будете использовать их в следующих упражнениях).
4. Закрой соединение.

Небольшой совет — выполняй все операции поочередно, от простой к более сложной, а не пытаясь сделать всё вместе и сразу. Это поможет в отладке твоих запросов.

In [10]:
# создаем соединение с бд
connection = sqlite3.connect('checking-logs.sqlite')

drop = """
DROP TABLE IF EXISTS datamart
"""
connection.execute(drop)

# создаем таблицу datamart
create_table_q = """
CREATE TABLE IF NOT EXISTS datamart AS
SELECT c.uid, c.labname AS labname, c.timestamp AS first_commit_ts, p.datetime AS first_view_ts
FROM checker c LEFT JOIN pageviews p ON c.uid = p.uid AND p.datetime = (SELECT MIN(datetime) FROM pageviews WHERE uid = c.uid)
WHERE c.uid LIKE 'user_%' AND c.status = 'ready' AND c.numTrials = 1 AND c.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
"""
connection.execute(create_table_q)

In [11]:
# создаем датафрейм datamart
datamart_q = "SELECT * FROM datamart"
datamart = pd.io.sql.read_sql(datamart_q, connection, parse_dates=['first_commit_ts', 'first_view_ts'])
display(datamart.info())
#display(datamart)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


None

In [12]:
# создаем датафрейм test
test_q = "SELECT * FROM datamart WHERE first_view_ts IS NOT NULL"
test = pd.io.sql.read_sql(test_q, connection, parse_dates=['first_commit_ts', 'first_view_ts'])
display(test.info())
#display(test)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.0+ KB


None

In [13]:
# создаем датафрейм control
control_q = "SELECT * FROM datamart WHERE first_view_ts IS NULL"
control = pd.io.sql.read_sql(control_q, connection, parse_dates=['first_commit_ts', 'first_view_ts'])

# заменяем пропущенные значения в control
control['first_view_ts'].fillna(test['first_view_ts'].mean(), inplace = True)

display(control.info())
#display(control.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    81 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.7+ KB


None

In [14]:
# заменяем пропущенные значения в control
control['first_view_ts'].fillna(test['first_view_ts'].mean(), inplace = True)

# сохраняем датафрейм в бд
test.to_sql('test', connection, index = False, if_exists='replace')
control.to_sql('control', connection, index = False, if_exists='replace')

query = """
SELECT * FROM test
"""
test = pd.io.sql.read_sql(query, connection)
display(test.head(5))

,uid,labname,first_commit_ts,first_view_ts
0,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447


In [15]:
query = """
SELECT * FROM control
"""
control = pd.io.sql.read_sql(query, connection)
display(control.head(5))

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,2020-04-27 00:40:05.761783
1,user_4,laba04,2020-04-17 11:33:17.366400,2020-04-27 00:40:05.761783
2,user_4,laba04s,2020-04-17 11:48:41.992466,2020-04-27 00:40:05.761783
3,user_2,laba04,2020-04-18 13:42:35.482008,2020-04-27 00:40:05.761783
4,user_2,laba04s,2020-04-18 13:51:22.291271,2020-04-27 00:40:05.761783


In [16]:
# закрываем соединение
connection.close()

### Task 4
На предыдущих этапах ты занимался просто подготовкой данных. Ты не получил никакой аналитической информации на основе имеющихся данных. Пришло время приступить к анализу. Помни, что, согласно нашему предположению, пользователи приступили бы к работе над лабораторными заданиями раньше, если бы они просмотрели ленту новостей? Это означает, что ключевой метрикой для нас является период времени (delta) между датой начала работы пользователя над лабораторным заданием (первого коммита) и сроком сдачи лабораторного задания. Мы будем смотреть на то, меняется ли она в зависимости от просмотра страницы.

Что тебе нужно сделать в этом упражнении:

1. Создай соединение с базой данных с помощью библиотеки `sqlite3`.
2. Получи схему таблицы `test`.
3. Получи только первые `10` строк таблицы `test`, чтобы проверить, как она выглядит.
4. Найди среди всех пользователей минимальное значение этой самой дельты (периода времени от даты первого коммита
пользователя до срока сдачи соответствующего лабораторного задания), используя только один запрос.
    - Для этого сджойни данные своей таблицы с данными таблицы `deadlines`.
    - Период времени должен отображаться в часах.
    - Не учитывай лабораторное задание `project1` с более длительным сроком сдачи, поскольку оно будет выделяться на фоне других.
    - Сохрани значение в датафрейме `df_min` с соответствующим `uid`.
5. Выполни те же самые операции для максимального значения дельты, используя только **один** запрос. Название итогового датафрейма — `df_max`.
6. Выполни те же самые операции для среднего значения дельты, используя только один запрос. На этот раз ваш итоговый датафрейм не должен включать столбец `uid`; имя датафрейма — `df_avg`.
7. Мы хотим проверить предположение о том, что у пользователей, посетивших ленту новостей всего несколько раз, период времени между датой первого коммита и сроком сдачи лабораторного задания меньше. Для этого тебе необходимо рассчитать коэффициент корреляции между количеством просмотров страниц и разницей между датами.
    - Используя только **один** запрос, создай таблицу со столбцами: `uid`, `avg_diff`, `pageviews`.
        - `uid` — это uid, существующие в таблице `test`.
        - `avg_diff` — среднее значение дельты (периода времени между датой первого коммита и сроком сдачи лабораторного задания) для каждого пользователя.
        - `pageviews` — количество посещений ленты новостей одним пользователем.
    - Не учитывай лабораторное задание `project1`.
    - Сохрани результаты в датафрейме `views_diff`.
    - Используй метод `corr()` библиотеки Pandas для вычисления коэффициента корреляции между количеством просмотров и значением дельты.
8. Закрой соединение.

In [17]:
# создаем соединение с бд
connection = sqlite3.connect('checking-logs.sqlite')

# ищем минимальное значение
min_query = """
SELECT uid, MIN(d.deadlines - strftime('%s', t.first_commit_ts)) / 3600 as "min_diff" FROM test t
LEFT JOIN deadlines d ON t.labname = d.labs WHERE labname != 'project1'
"""
df_min = pd.io.sql.read_sql(min_query, connection)
display(df_min)

,uid,min_diff
0,user_25,2


In [18]:
# ищем максимальное значение
max_query = """
SELECT uid, MAX(d.deadlines - strftime('%s', t.first_commit_ts)) / 3600 as "max_diff" FROM test t
LEFT JOIN deadlines d ON t.labname = d.labs WHERE labname != 'project1'
"""
df_max = pd.io.sql.read_sql(max_query, connection)
display(df_max)

,uid,max_diff
0,user_30,202


In [19]:
# ищем среднее значение
avg_query = """
SELECT AVG((d.deadlines - strftime('%s', t.first_commit_ts)) / 3600) as "avg_diff" FROM test t
LEFT JOIN deadlines d ON t.labname = d.labs WHERE labname != 'project1'
"""
df_avg = pd.io.sql.read_sql(avg_query, connection)
display(df_avg)

,avg_diff
0,89.125


In [20]:
# рассчитываем коэффициент корреляции между количеством просмотров страниц и разницей между датами
drop = """
DROP TABLE IF EXISTS views_diff;
"""
connection.execute(drop)

views_diff_q = """
SELECT t.uid, 

AVG(CEIL((julianday(first_commit_ts) - julianday(DATETIME(deadlines, 'unixepoch')))*24)) AS avg_diff,
cnt as pageviews
FROM test t, pageviews pw
LEFT JOIN deadlines dl
ON t.labname = dl.labs
LEFT JOIN
    (SELECT uid as uid, 
    COUNT(uid) as cnt
    FROM pageviews pw
    GROUP BY uid) AS p1
ON t.uid = p1.uid
WHERE labname != 'project1'
GROUP BY t.uid
ORDER BY CAST(SUBSTR(t.uid, 6) AS NUMERIC);
"""
views_diff = pd.io.sql.read_sql(views_diff_q, connection)
display(views_diff.corr(numeric_only=True))

,avg_diff,pageviews
avg_diff,1.000000,-0.279736
pageviews,-0.279736,1.000000


In [21]:
# закрываем соединение
connection.close()

### Task 5
Итак... давай, наконец, выясним, повлияло ли посещение ленты новостей на поведение учащихся. 
Приступили ли они в итоге к работе над лабораторным заданием раньше? Помни, что у нас есть две подготовленные 
таблицы в базе данных: `test` и `control`. Мы выполним нечто, похожее на A/B-тестирование.
Чтобы обнаружить эффект, нам нужно вычислить значение дельты (период времени между датой первого коммита и
сроком сдачи лабораторного задания) до того момента, когда учащиеся впервые посетили страницу с лентой новостей, и после этого.
Мы должны сделать то же самое и для контрольной группы.

Другими словами, каждый пользователь из тестовой таблицы имеет свою собственную временную метку для
первого посещения новостной ленты. Мы хотим вычислить среднее значение дельты 
(разницу между датой первого коммита и сроком сдачи) до этой временной метки и после нее. 
Мы сделаем то же самое для пользователей в контрольной группе. Ты можешь сказать: «Но они вообще не посещали ленту новостей». 
Это так, и ранее мы решили использовать среднюю временную метку первого просмотра пользователями тестовой группы для пользователей контрольной группы.

Если значение дельты перед первым посещением ленты новостей значительно отличается от этого показателя после первого 
посещения в тестовой группе, и мы не видим аналогичного эффекта в контрольной группе, значит, создание страницы 
с новостной лентой было отличной идеей. Мы можем распространить эту практику на всю группу.


Подробное описание:
1. Создай соединение с базой данных с помощью библиотеки `sqlite3`.
2. Используя только **один** запрос для каждой из групп, создай два датафрейма: `test_results` и `control_results` со столбцами `time` и `avg_diff` и только двумя строками.
    - `times` должно иметь значения `before` и `after`.
    - `avg_diff` содержит среднее значение дельты для всех пользователей за период времени до первого посещения ленты новостей каждым из них и после этого.
    - Учитываются только те пользователи, для которых имеются наблюдения и `before`, и `after`.
3. Мы по-прежнему не используем лабораторное задание 'project1'.
4. Закрой соединение.
5. Дайте ответ: оказалось ли предположение верным и влияет ли наличие страницы с новостной лентой на поведение учащихся?

In [22]:
# создаем соединение с бд
connection = sqlite3.connect('checking-logs.sqlite')

# создаем датафрейм test_results
test_res_q = """
WITH test_avg as (
SELECT uid, labname, first_commit_ts, first_view_ts, DATETIME(dl.deadlines, 'unixepoch') as deadlines,
AVG((strftime('%s', t.first_commit_ts) - dl.deadlines) / 3600) AS delta,
(CASE
WHEN strftime('%s', t.first_commit_ts) - strftime('%s', t.first_view_ts) > 0 THEN 'after'
WHEN strftime('%s', t.first_commit_ts) - strftime('%s', t.first_view_ts) < 0 THEN 'before'
ELSE null
END) as times
FROM test t LEFT JOIN deadlines dl ON t.labname = dl.labs WHERE t.labname != 'project1'
GROUP BY uid, times
)
SELECT times, AVG(delta) as avg_diff FROM test_avg
where uid IN(select uid from test_avg where times = 'before') AND uid IN(select uid from test_avg where times = 'after')
GROUP BY times
"""
test_res = pd.io.sql.read_sql(test_res_q, connection)
display(test_res)

,times,avg_diff
0,after,-99.523810
1,before,-66.047619


In [23]:
# создаем датафрейм control_results
ctrl_res_q = """
WITH ctrl_avg as (
SELECT uid, labname, first_commit_ts, first_view_ts, DATETIME(dl.deadlines, 'unixepoch') as deadlines,
AVG((strftime('%s', t.first_commit_ts) - dl.deadlines) / 3600) AS delta,
(CASE
WHEN strftime('%s', t.first_commit_ts) - strftime('%s', t.first_view_ts) > 0 THEN 'after'
WHEN strftime('%s', t.first_commit_ts) - strftime('%s', t.first_view_ts) < 0 THEN 'before'
ELSE null
END) as times
FROM control t LEFT JOIN deadlines dl ON t.labname = dl.labs WHERE t.labname != 'project1'
GROUP BY uid, times
)
SELECT times, AVG(delta) as avg_diff FROM ctrl_avg
where uid IN(select uid from ctrl_avg where times = 'before') AND uid IN(select uid from ctrl_avg where times = 'after')
GROUP BY times
"""
ctrl_res = pd.io.sql.read_sql(ctrl_res_q, connection)
display(ctrl_res)

,times,avg_diff
0,after,-99.322222
1,before,-98.033333


### Используя тезис из условия:
"Если значение дельты перед первым посещением ленты новостей значительно отличается от этого показателя после первого 
посещения в тестовой группе, и мы не видим аналогичного эффекта в контрольной группе, значит, создание страницы 
с новостной лентой было отличной идеей. Мы можем распространить эту практику на всю группу."
### Делаем вывод, что наличие страницы с новостной лентой влияет на поведение учащихся